In [87]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import glob
from scipy.optimize import fsolve
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.interpolate import splev
from scipy.interpolate import splrep
import colorsys
import networkx as nx
import random
import math
import matplotlib.style
import matplotlib as mpl
import pandas as pd
import graph_tool.all as gt
import seaborn as sns
import multiprocessing as mp
import json
import sys
from scipy.stats import hypergeom
from tqdm import tqdm
import networkx.algorithms.community as nx_comm
import NetworkMetrics as metrics
from scipy import stats
import csv
from collections import defaultdict
from matplotlib.colors import ListedColormap, Normalize
import seaborn as sns
import copy
import random
from cmapPy.pandasGEXpress import parse
from scipy.stats import pearsonr
import matplotlib.style
import matplotlib as mpl
from scipy import stats
import bisect
import ast


mpl.style.use('default')
mpl.style.use('classic')



In [89]:

path_interactome = "./data/PPI_2022.csv"
G = nx.from_pandas_edgelist(pd.read_csv(path_interactome), 'HGNC_Symbol.1', 'HGNC_Symbol.2')
print("interactome2022 - ppi")

self_loops = [(u, v) for u, v in G.edges() if u == v]
G.remove_edges_from(self_loops)

connected_components = list(nx.connected_components(G))
lcc = max(len(component) for component in connected_components)



interactome2022 - ppi


In [91]:
path = "./data/Gene_hallmarks.csv"
gene_hallmarks_extended = pd.read_csv(path)
all_hallmarks = gene_hallmarks_extended['aging_mechanisms_group'].unique()
gene_hallmarks_extended = gene_hallmarks_extended[gene_hallmarks_extended['confidence'] <= 4]

In [93]:
aging_mechanisms_groups = ['Exhaustion of stem cells',
              'Altered intercellular communication',
                        'Epigenetic alterations',
                'Mitochondrial dysfunction',
                 'Loss of proteostasis',
                      'Changes in the extracellular matrix structure',
                'Deregulated nutrient sensing',
                    'Genomic instability',
                 'Cell senescence',
                 'Disabled macroautophagy',
                  'Telomere attrition'        ]

In [95]:
drug_evidence_short = pd.read_csv("./Output/Proximity_drugs_log_binning_level1.csv")

In [97]:
path = "./data/age-related-changes.tsv"
age_related_changes = pd.read_csv(path, delimiter = '\t')
age_related_changes = age_related_changes[~(age_related_changes['p value'] == '>0.01')]
age_related_changes = age_related_changes[age_related_changes['model organism'] == 'human']

UDEG = []
DDEG = []
count = 0
for gene in set(age_related_changes['HGNC']):
    A = age_related_changes[age_related_changes['change type'] == 'increased gene expression']
    B = age_related_changes[age_related_changes['change type'] == 'decreased gene expression']
    if len(A[A['HGNC'] == gene]) > 0:
        UDEG.append(gene)
    if len(B[B['HGNC'] == gene]) > 0:
        DDEG.append(gene)


In [98]:
#random gene signature
gene_signature = pd.DataFrame(G.nodes(), columns=['gene'])
gene_signature['sig'] = [random.choice([1, -1]) for _ in range(len(G.nodes()))]



In [99]:
gene_signature

,gene,sig
0,A1BG,-1
1,ABCC6,-1
2,ANXA7,-1
3,CDKN1A,1
4,CRISP3,-1
...,...,...
18218,USP17L7,-1
18219,USP41,-1
18220,UTS2R,-1
18221,VPREB3,1


In [103]:
test_drugs = set(drug_evidence_short['drug'])
drug_evidence_pAGE = pd.DataFrame(test_drugs, columns=['drug'])

level = 1
path = "./data/Gene_hallmarks.csv"
gene_hallmarks_extended = pd.read_csv(path)
all_hallmarks = gene_hallmarks_extended['aging_mechanisms'].unique()
gene_hallmarks_extended = gene_hallmarks_extended[gene_hallmarks_extended['confidence'] <= level]
for idx_amg, amg in enumerate(aging_mechanisms_groups):
    pAGE = []
    pAGE_z = []
    am_genes = set(gene_hallmarks_extended[gene_hallmarks_extended['aging_mechanisms_group'] == amg]['GeneId'])
    if len(am_genes) == 0:
            pAGE = [-2 for i in test_drugs]
            pAGE_z = [0 for i in test_drugs]
            drug_evidence_pAGE['Cmap pAGE ' + amg+ ' level' + str(level)] = pAGE
            drug_evidence_pAGE['Cmap pAGE ' + amg+ ' level' + str(level) + ' z_score'] = pAGE_z
            continue
    
    for idxx, compound in enumerate(test_drugs):   
        
        d_count = 0
        module_count = 0
        
        for gene in am_genes:
                
            g_sig = list(gene_signature[gene_signature['gene'] == gene]['sig'])[0]
            
                
            if gene in UDEG and g_sig < 0:
                d_count = d_count +1
            elif gene in DDEG and g_sig > 0:
                d_count = d_count +1
            elif gene in UDEG and g_sig > 0:
                d_count = d_count -1
            elif gene in DDEG and g_sig < 0:
                d_count = d_count -1
            
            module_count = module_count + 1




        if module_count != 0:
            pAGE_val = d_count / module_count
            pAGE.append(pAGE_val)
            sigma = 1 / np.sqrt(module_count)
            pAGE_z.append(pAGE_val / sigma)
        else:
            pAGE.append(0)
            pAGE_z.append(0)
            
            
        print(level, idx_amg, idxx, module_count)
    drug_evidence_pAGE['Cmap pAGE ' + amg+ ' level' + str(level)] = pAGE
    drug_evidence_pAGE['Cmap pAGE ' + amg+ ' level' + str(level) + ' z_score'] = pAGE_z


1 1 0 9
1 1 1 9
1 1 2 9
1 1 3 9
1 1 4 9
1 2 0 8
1 2 1 8
1 2 2 8
1 2 3 8
1 2 4 8
1 3 0 3
1 3 1 3
1 3 2 3
1 3 3 3
1 3 4 3
1 4 0 3
1 4 1 3
1 4 2 3
1 4 3 3
1 4 4 3
1 6 0 10
1 6 1 10
1 6 2 10
1 6 3 10
1 6 4 10
1 7 0 3
1 7 1 3
1 7 2 3
1 7 3 3
1 7 4 3
1 8 0 5
1 8 1 5
1 8 2 5
1 8 3 5
1 8 4 5
1 10 0 1
1 10 1 1
1 10 2 1
1 10 3 1
1 10 4 1


1 8 1340 2
1 8 1355 2
1 8 1357 2
1 8 1366 2
1 8 1378 2
1 8 1382 2
1 8 1389 2
1 8 1391 2
1 8 1392 2
1 8 1394 2
1 8 1397 2
1 8 1406 2
1 8 1407 2
1 8 1408 2
1 8 1411 2
1 8 1417 2
1 8 1419 2
1 8 1420 2
1 8 1422 2
1 8 1424 2
1 8 1426 2
1 8 1428 2
1 8 1431 2
1 8 1442 2
1 8 1444 2
1 8 1449 2
1 8 1453 2
1 8 1454 2
1 8 1467 2
1 8 1476 2
1 8 1480 2
1 8 1483 2
1 8 1489 2
1 8 1490 2
1 8 1512 2
1 8 1514 2
1 8 1532 2
1 8 1533 2
1 8 1538 2
1 8 1543 2
1 8 1544 2
1 8 1553 2
1 8 1556 2
1 8 1561 2
1 8 1567 2
1 8 1576 2
1 8 1579 2
1 8 1586 2
1 8 1588 2
1 8 1591 2
1 8 1592 2
1 8 1602 2
1 8 1607 2
1 8 1608 2
1 8 1615 2
1 8 1622 2
1 8 1623 2
1 8 1631 2
1 8 1634 2
1 8 1636 2
1 8 1637 2
1 8 1639 2
1 8 1652 2
1 8 1653 2
1 8 1656 2
1 8 1663 2
1 8 1676 2
1 8 1679 2
1 8 1694 2
1 8 1708 2
1 8 1734 2
1 8 1736 2
1 8 1737 2
1 8 1740 2
1 8 1743 2
1 8 1749 2
1 8 1750 2
1 8 1753 2
1 8 1756 2
1 8 1758 2
1 8 1759 2
1 8 1769 2
1 8 1780 2
1 8 1789 2
1 8 1791 2
1 8 1797 2
1 8 1810 2
1 8 1811 2
1 8 1813 2
1 8 1815 2
1 8 1818 2

1 8 4967 2
1 8 4968 2
1 8 4971 2
1 8 4973 2
1 8 4980 2
1 8 4981 2
1 8 4988 2
1 8 4991 2
1 8 4998 2
1 8 5000 2
1 8 5002 2
1 8 5004 2
1 8 5005 2
1 8 5006 2
1 8 5012 2
1 8 5021 2
1 8 5024 2
1 8 5026 2
1 8 5028 2
1 8 5029 2
1 8 5031 2
1 8 5032 2
1 8 5038 2
1 8 5039 2
1 8 5040 2
1 8 5044 2
1 8 5051 2
1 8 5053 2
1 8 5054 2
1 8 5057 2
1 8 5065 2
1 8 5070 2
1 8 5072 2
1 8 5073 2
1 8 5075 2
1 8 5076 2
1 8 5078 2
1 8 5082 2
1 8 5085 2
1 8 5092 2
1 8 5094 2
1 8 5096 2
1 8 5102 2
1 8 5103 2
1 8 5104 2
1 8 5105 2
1 8 5124 2
1 8 5126 2
1 8 5130 2
1 8 5131 2
1 8 5136 2
1 8 5138 2
1 8 5145 2
1 8 5146 2
1 8 5150 2
1 8 5152 2
1 8 5155 2
1 8 5156 2
1 8 5162 2
1 8 5176 2
1 8 5182 2
1 8 5190 2
1 8 5191 2
1 8 5196 2
1 8 5198 2
1 8 5200 2
1 8 5201 2
1 8 5202 2
1 8 5204 2
1 8 5205 2
1 8 5212 2
1 8 5216 2
1 8 5220 2
1 8 5221 2
1 8 5230 2
1 8 5231 2
1 8 5234 2
1 8 5240 2
1 8 5241 2
1 8 5242 2
1 8 5245 2
1 8 5254 2
1 8 5255 2
1 8 5257 2
1 8 5264 2
1 8 5266 2
1 8 5269 2
1 8 5270 2
1 8 5275 2
1 8 5295 2
1 8 5300 2

1 10 1927 0
1 10 1928 0
1 10 1931 0
1 10 1935 0
1 10 1938 0
1 10 1944 0
1 10 1948 0
1 10 1951 0
1 10 1957 0
1 10 1959 0
1 10 1963 0
1 10 1968 0
1 10 1969 0
1 10 1975 0
1 10 1984 0
1 10 1985 0
1 10 1987 0
1 10 1989 0
1 10 1994 0
1 10 1995 0
1 10 2002 0
1 10 2013 0
1 10 2016 0
1 10 2018 0
1 10 2019 0
1 10 2020 0
1 10 2022 0
1 10 2025 0
1 10 2031 0
1 10 2036 0
1 10 2037 0
1 10 2039 0
1 10 2045 0
1 10 2056 0
1 10 2058 0
1 10 2074 0
1 10 2100 0
1 10 2101 0
1 10 2102 0
1 10 2104 0
1 10 2105 0
1 10 2112 0
1 10 2123 0
1 10 2124 0
1 10 2125 0
1 10 2126 0
1 10 2135 0
1 10 2138 0
1 10 2141 0
1 10 2145 0
1 10 2148 0
1 10 2150 0
1 10 2151 0
1 10 2152 0
1 10 2153 0
1 10 2161 0
1 10 2164 0
1 10 2166 0
1 10 2169 0
1 10 2173 0
1 10 2176 0
1 10 2180 0
1 10 2187 0
1 10 2189 0
1 10 2192 0
1 10 2193 0
1 10 2198 0
1 10 2199 0
1 10 2206 0
1 10 2207 0
1 10 2210 0
1 10 2211 0
1 10 2212 0
1 10 2213 0
1 10 2214 0
1 10 2216 0
1 10 2218 0
1 10 2223 0
1 10 2224 0
1 10 2226 0
1 10 2229 0
1 10 2230 0
1 10 2232 0
1 10

1 10 5201 0
1 10 5202 0
1 10 5204 0
1 10 5205 0
1 10 5212 0
1 10 5216 0
1 10 5220 0
1 10 5221 0
1 10 5230 0
1 10 5231 0
1 10 5234 0
1 10 5240 0
1 10 5241 0
1 10 5242 0
1 10 5245 0
1 10 5254 0
1 10 5255 0
1 10 5257 0
1 10 5264 0
1 10 5266 0
1 10 5269 0
1 10 5270 0
1 10 5275 0
1 10 5295 0
1 10 5300 0
1 10 5302 0
1 10 5308 0
1 10 5313 0
1 10 5321 0
1 10 5333 0
1 10 5335 0
1 10 5349 0
1 10 5350 0
1 10 5351 0
1 10 5362 0
1 10 5363 0
1 10 5364 0
1 10 5367 0
1 10 5371 0
1 10 5372 0
1 10 5384 0
1 10 5401 0
1 10 5412 0
1 10 5452 0
1 10 5458 0
1 10 5473 0
1 10 5481 0
1 10 5482 0
1 10 5484 0
1 10 5511 0
1 10 5527 0
1 10 5551 0
1 10 5552 0
1 10 5553 0
1 10 5554 0
1 10 5558 0
1 10 5571 0
1 10 5580 0
1 10 5583 0
1 10 5585 0
1 10 5589 0
1 10 5593 0
1 10 5596 0
1 10 5597 0
1 10 5599 0
1 10 5601 0
1 10 5605 0
1 10 5606 0
1 10 5609 0
1 10 5623 0
1 10 5626 0
1 10 5632 0
1 10 5641 0
1 10 5642 0
1 10 5644 0
1 10 5645 0
1 10 5647 0
1 10 5649 0
1 10 5654 0
1 10 5657 0
1 10 5658 0
1 10 5659 0
1 10 5669 0
1 10

...

5 8 5589 31
5 8 5593 31
5 8 5596 31
5 8 5597 31
5 8 5599 31
5 8 5601 31
5 8 5605 31
5 8 5606 31
5 8 5609 31
5 8 5623 31
5 8 5626 31
5 8 5632 31
5 8 5641 31
5 8 5642 31
5 8 5644 31
5 8 5645 31
5 8 5647 31
5 8 5649 31
5 8 5654 31
5 8 5657 31
5 8 5658 31
5 8 5659 31
5 8 5669 31
5 8 5670 31
5 8 5675 31
5 8 5676 31
5 8 5677 31
5 8 5678 31
5 8 5679 31
5 8 5681 31
5 8 5684 31
5 8 5688 31
5 8 5690 31
5 8 5692 31
5 8 5693 31
5 8 5696 31
5 8 5697 31
5 8 5700 31
5 8 5703 31
5 8 5705 31
5 8 5706 31
5 8 5708 31
5 8 5710 31
5 8 5713 31
5 8 5716 31
5 8 5720 31
5 8 5721 31
5 8 5724 31
5 8 5727 31
5 8 5730 31
5 8 5731 31
5 8 5733 31
5 8 5734 31
5 8 5736 31
5 8 5740 31
5 8 5743 31
5 8 5748 31
5 8 5751 31
5 8 5758 31
5 8 5759 31
5 8 5765 31
5 8 5770 31
5 8 5778 31
5 8 5783 31
5 8 5784 31
5 8 5790 31
5 8 5792 31
5 8 5801 31
5 8 5807 31
5 8 5820 31
5 8 5824 31
5 8 5866 31
5 8 5869 31
5 8 5870 31
5 8 5877 31
5 8 5880 31
5 8 5888 31
5 8 5893 31
5 8 5899 31
5 8 5904 31
5 8 5911 31
5 8 5913 31
5 8 5920 31
5 8 

5 9 2372 20
5 9 2381 20
5 9 2403 20
5 9 2408 20
5 9 2420 20
5 9 2442 20
5 9 2455 20
5 9 2468 20
5 9 2471 20
5 9 2480 20
5 9 2492 20
5 9 2497 20
5 9 2498 20
5 9 2509 20
5 9 2533 20
5 9 2535 20
5 9 2547 20
5 9 2548 20
5 9 2555 20
5 9 2558 20
5 9 2562 20
5 9 2565 20
5 9 2568 20
5 9 2570 20
5 9 2572 20
5 9 2574 20
5 9 2583 20
5 9 2590 20
5 9 2591 20
5 9 2595 20
5 9 2596 20
5 9 2597 20
5 9 2600 20
5 9 2611 20
5 9 2613 20
5 9 2615 20
5 9 2620 20
5 9 2625 20
5 9 2631 20
5 9 2635 20
5 9 2642 20
5 9 2646 20
5 9 2657 20
5 9 2661 20
5 9 2665 20
5 9 2670 20
5 9 2675 20
5 9 2676 20
5 9 2687 20
5 9 2691 20
5 9 2724 20
5 9 2729 20
5 9 2730 20
5 9 2734 20
5 9 2735 20
5 9 2736 20
5 9 2771 20
5 9 2784 20
5 9 2789 20
5 9 2804 20
5 9 2806 20
5 9 2807 20
5 9 2819 20
5 9 2829 20
5 9 2836 20
5 9 2838 20
5 9 2840 20
5 9 2844 20
5 9 2845 20
5 9 2854 20
5 9 2864 20
5 9 2871 20
5 9 2872 20
5 9 2876 20
5 9 2878 20
5 9 2886 20
5 9 2902 20
5 9 2920 20
5 9 2934 20
5 9 2938 20
5 9 2943 20
5 9 2945 20
5 9 2949 20
5 9 

5 9 5700 20
5 9 5703 20
5 9 5705 20
5 9 5706 20
5 9 5708 20
5 9 5710 20
5 9 5713 20
5 9 5716 20
5 9 5720 20
5 9 5721 20
5 9 5724 20
5 9 5727 20
5 9 5730 20
5 9 5731 20
5 9 5733 20
5 9 5734 20
5 9 5736 20
5 9 5740 20
5 9 5743 20
5 9 5748 20
5 9 5751 20
5 9 5758 20
5 9 5759 20
5 9 5765 20
5 9 5770 20
5 9 5778 20
5 9 5783 20
5 9 5784 20
5 9 5790 20
5 9 5792 20
5 9 5801 20
5 9 5807 20
5 9 5820 20
5 9 5824 20
5 9 5866 20
5 9 5869 20
5 9 5870 20
5 9 5877 20
5 9 5880 20
5 9 5888 20
5 9 5893 20
5 9 5899 20
5 9 5904 20
5 9 5911 20
5 9 5913 20
5 9 5920 20
5 9 5927 20
5 9 5929 20
5 9 5931 20
5 9 5933 20
5 9 5939 20
5 9 5940 20
5 9 5944 20
5 9 5945 20
5 9 5947 20
5 9 5953 20
5 9 5955 20
5 9 5959 20
5 9 5960 20
5 9 5966 20
5 9 5968 20
5 9 5969 20
5 9 5973 20
5 9 5977 20
5 9 5984 20
5 9 5988 20
5 9 5990 20
5 9 5991 20
5 9 5993 20
5 9 5999 20
5 9 6001 20
5 9 6003 20
5 9 6005 20
5 9 6007 20
5 9 6009 20
5 9 6011 20
5 9 6014 20
5 9 6015 20
5 9 6016 20
5 9 6017 20
5 9 6020 20
5 9 6023 20
5 9 6024 20
5 9 

5 10 2359 19
5 10 2362 19
5 10 2372 19
5 10 2381 19
5 10 2403 19
5 10 2408 19
5 10 2420 19
5 10 2442 19
5 10 2455 19
5 10 2468 19
5 10 2471 19
5 10 2480 19
5 10 2492 19
5 10 2497 19
5 10 2498 19
5 10 2509 19
5 10 2533 19
5 10 2535 19
5 10 2547 19
5 10 2548 19
5 10 2555 19
5 10 2558 19
5 10 2562 19
5 10 2565 19
5 10 2568 19
5 10 2570 19
5 10 2572 19
5 10 2574 19
5 10 2583 19
5 10 2590 19
5 10 2591 19
5 10 2595 19
5 10 2596 19
5 10 2597 19
5 10 2600 19
5 10 2611 19
5 10 2613 19
5 10 2615 19
5 10 2620 19
5 10 2625 19
5 10 2631 19
5 10 2635 19
5 10 2642 19
5 10 2646 19
5 10 2657 19
5 10 2661 19
5 10 2665 19
5 10 2670 19
5 10 2675 19
5 10 2676 19
5 10 2687 19
5 10 2691 19
5 10 2724 19
5 10 2729 19
5 10 2730 19
5 10 2734 19
5 10 2735 19
5 10 2736 19
5 10 2771 19
5 10 2784 19
5 10 2789 19
5 10 2804 19
5 10 2806 19
5 10 2807 19
5 10 2819 19
5 10 2829 19
5 10 2836 19
5 10 2838 19
5 10 2840 19
5 10 2844 19
5 10 2845 19
5 10 2854 19
5 10 2864 19
5 10 2871 19
5 10 2872 19
5 10 2876 19
5 10 2878 19

5 10 5412 19
5 10 5452 19
5 10 5458 19
5 10 5473 19
5 10 5481 19
5 10 5482 19
5 10 5484 19
5 10 5511 19
5 10 5527 19
5 10 5551 19
5 10 5552 19
5 10 5553 19
5 10 5554 19
5 10 5558 19
5 10 5571 19
5 10 5580 19
5 10 5583 19
5 10 5585 19
5 10 5589 19
5 10 5593 19
5 10 5596 19
5 10 5597 19
5 10 5599 19
5 10 5601 19
5 10 5605 19
5 10 5606 19
5 10 5609 19
5 10 5623 19
5 10 5626 19
5 10 5632 19
5 10 5641 19
5 10 5642 19
5 10 5644 19
5 10 5645 19
5 10 5647 19
5 10 5649 19
5 10 5654 19
5 10 5657 19
5 10 5658 19
5 10 5659 19
5 10 5669 19
5 10 5670 19
5 10 5675 19
5 10 5676 19
5 10 5677 19
5 10 5678 19
5 10 5679 19
5 10 5681 19
5 10 5684 19
5 10 5688 19
5 10 5690 19
5 10 5692 19
5 10 5693 19
5 10 5696 19
5 10 5697 19
5 10 5700 19
5 10 5703 19
5 10 5705 19
5 10 5706 19
5 10 5708 19
5 10 5710 19
5 10 5713 19
5 10 5716 19
5 10 5720 19
5 10 5721 19
5 10 5724 19
5 10 5727 19
5 10 5730 19
5 10 5731 19
5 10 5733 19
5 10 5734 19
5 10 5736 19
5 10 5740 19
5 10 5743 19
5 10 5748 19
5 10 5751 19
5 10 5758 19

In [109]:
drug_evidence_pAGE.to_csv("./Output/pAGE_drugs_log_binning_level1.csv")